# MedReg Parsing erste Schritte

Das Ziel: einen Teil der html-Daten aus dem Scraping des MedReg parsen und in Pandas testweise auswerten. 

Schritt 1: wir erstellen aus den gesamten Scraping-Daten einen neuen Ordner *MedReg_Scraped_data_20k* mit den Daten bis Eintrag 20'000 (19'789 Files).

Wir sollten unter den 19789 Seiten eigentlich zwei Arten von Dateien haben. Solche mit Daten im html-File (Code 200) und solche bei denen das html-File nur aus einer Fehlermeldung besteht, da zur abgefragten Personen-ID kein Eintrag existiert (Code 500). 


In [20]:
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import re
from tqdm import tqdm


In [21]:
path = 'MedReg_Scraped_Data_20K/'

file_list = os.listdir(path)
len(file_list)

19789

Wir lesen ein File, von dem wir wissen, dass es Daten enthält. 

In [22]:
file = open(path + file_list[4805], 'r')
text = file.read()
soup = BeautifulSoup(text, 'html.parser')
print(soup)

<?xml version="1.0" encoding="utf-8"?>
<div>
<div style="min-height:51px">
<h3>Blaschegg-Honsalek, Irma <img class="geschlecht-icon" src="../../Images/venus-solid.png"/></h3>
<div class="favorit detail"> </div>
<div style="clear:left"></div>
</div>
<p>Nationalität: Deutschland (DE)<br/>Sprachkenntnisse: Deutsch<br/>GLN: 7601000677815<a class="info" title="Global Location Number: eindeutige Nummer einer Medizinalperson (früher EAN) "> </a><br/>UID: CHE103519026<a class="info" title="UID = Unternehmens-Identifikationsnummer. Die UID dient der eindeutigen Identifikation von Unternehmen und wird jeder organisatorischen oder institutionellen Einheit, die aus rechtlichen, administrativen oder statistischen Gründen identifiziert werden muss (UID-Einheit), zugewiesen. Die UID ist die Nachfolge für die MWST-Nr. und die Handelsregisternummer und wird auch von weiteren Verwaltungsregistern, wie z.B. den AHV-Ausgleichskassen geführt."> </a></p>
</div>
<div class="right">
<div id="noresult_overlay"

In [7]:
name = soup.find('h3').text
name

'Blaschegg-Honsalek, Irma\xa0'

In [8]:
name = soup.find('h3').text.replace('\xa0','')
name

'Blaschegg-Honsalek, Irma'

Nächste Aufgabe: das Geschlecht

In [144]:
geschlecht = get_gender(soup)#soup.find('h3').text.replace('*/venus-solid.png', 'w')
print(geschlecht)

Blaschegg-Honsalek, Irma 


Der Name ist jetzt schön. Doch das Geschlechtslogo wird nicht erkannt. 

Nochmals:


In [7]:
name_geschlecht = soup.find('h3')
name_geschlecht
                                        

<h3>Blaschegg-Honsalek, Irma <img class="geschlecht-icon" src="../../Images/venus-solid.png"/></h3>

In [ ]:
geschlecht = name_geschlecht(replace)

In [154]:
geschlecht = re.sub(r'venus', 'w', name_geschlecht)
print(geschlecht)

TypeError: expected string or bytes-like object

In [114]:
geschlecht = re.search(r'(?=-solid.png)\w-', str(name_geschlecht))
print(geschlecht)

#\w\b(?=-solid) funktioniert auch nicht
#m = re.search(r'(?<=-)\w+', 'spam-egg')

None


In [9]:
regex_sex = r"^.+$-solid.png"
re.search(regex_sex, str(name_geschlecht))

In [ ]:
Neuer Anlauf, mit einer Funktion:

In [27]:
def get_gender(soup):
    img_source = soup.find('h3').find("img")["src"]
    if "venus" in img_source:
        return "w"
    else:
        return "m"

In [28]:
get_gender(soup)

'w'

Zum nächsten Tag:

In [52]:
nation = soup.find('p').text
nation

'Nationalität: Deutschland (DE)Sprachkenntnisse: DeutschGLN: 7601000677815\xa0UID: CHE103519026\xa0'

Ok, damit wollen wir mal leben, bevor wir uns auch hier noch ewig aufhalten. 

Weiter an die Daten über den Beruf:

In [75]:
beruf = soup.find_all('<li id="1">')
beruf

[]

Auch hier wird's schwierig. Nochmals kurz von vorne, gemäss bs4-Dokumentation. 

In [76]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(text, 'html.parser')
print(soup.prettify())


<?xml version="1.0" encoding="utf-8"?>
<div>
 <div style="min-height:51px">
  <h3>
   Blaschegg-Honsalek, Irma
   <img class="geschlecht-icon" src="../../Images/venus-solid.png"/>
  </h3>
  <div class="favorit detail">
  </div>
  <div style="clear:left">
  </div>
 </div>
 <p>
  Nationalität: Deutschland (DE)
  <br/>
  Sprachkenntnisse: Deutsch
  <br/>
  GLN: 7601000677815
  <a class="info" title="Global Location Number: eindeutige Nummer einer Medizinalperson (früher EAN) ">
  </a>
  <br/>
  UID: CHE103519026
  <a class="info" title="UID = Unternehmens-Identifikationsnummer. Die UID dient der eindeutigen Identifikation von Unternehmen und wird jeder organisatorischen oder institutionellen Einheit, die aus rechtlichen, administrativen oder statistischen Gründen identifiziert werden muss (UID-Einheit), zugewiesen. Die UID ist die Nachfolge für die MWST-Nr. und die Handelsregisternummer und wird auch von weiteren Verwaltungsregistern, wie z.B. den AHV-Ausgleichskassen geführt.">
  </a>
 <

In [80]:
soup.p

<p>Nationalität: Deutschland (DE)<br/>Sprachkenntnisse: Deutsch<br/>GLN: 7601000677815<a class="info" title="Global Location Number: eindeutige Nummer einer Medizinalperson (früher EAN) "> </a><br/>UID: CHE103519026<a class="info" title="UID = Unternehmens-Identifikationsnummer. Die UID dient der eindeutigen Identifikation von Unternehmen und wird jeder organisatorischen oder institutionellen Einheit, die aus rechtlichen, administrativen oder statistischen Gründen identifiziert werden muss (UID-Einheit), zugewiesen. Die UID ist die Nachfolge für die MWST-Nr. und die Handelsregisternummer und wird auch von weiteren Verwaltungsregistern, wie z.B. den AHV-Ausgleichskassen geführt."> </a></p>

In [82]:
soup.a

<a class="info" title="Global Location Number: eindeutige Nummer einer Medizinalperson (früher EAN) "> </a>

In [16]:
soup.find_all('a')

[<a class="info" title="Global Location Number: eindeutige Nummer einer Medizinalperson (früher EAN) "> </a>,
 <a class="info" title="UID = Unternehmens-Identifikationsnummer. Die UID dient der eindeutigen Identifikation von Unternehmen und wird jeder organisatorischen oder institutionellen Einheit, die aus rechtlichen, administrativen oder statistischen Gründen identifiziert werden muss (UID-Einheit), zugewiesen. Die UID ist die Nachfolge für die MWST-Nr. und die Handelsregisternummer und wird auch von weiteren Verwaltungsregistern, wie z.B. den AHV-Ausgleichskassen geführt."> </a>,
 <a class="info" title="Datum der Anerkennung des ausländischen Titels durch die Schweiz: Datum nach 1.6.2002 Anerkennung für EU/EFTA-Bürger durch die Bundesbehörden. Datum vor 1.6.2002 Bestätigung durch den Kanton."> </a>,
 <a class="info" title="Erteilt, aktiv = gültige Berufsausübungsbewilligung für diesen Kanton, aktiv / Erteilt, inaktiv = gültige Berufsausübungsbewilligung für diesen Kanton, inaktiv /

In [73]:
soup.find_all("ol")

[<ol style="list-style-type: circle">
 <li>Zürich
                 (2012), aktiv, MedBG, privatwirtschaftliche Berufsausübung in eigener fachlicher Verantwortung</li>
 </ol>,
 <ol class="address"><li><span class="outdent">Bewilligungskanton: Zürich</span></li><li class="address-item" title="Dufourstr. 169,8008 Zürich,ZH"><span class="outdent">A.
             </span><div>-<br/>Dufourstr. 169<br/>8008 Zürich<br/>UID: <a class="info" title="UID der juristischen Person/Betrieb welche dieser Adresse entspricht."> </a></div></li></ol>]

Ok, uns interessiert das letzte ol-Element mit der Adressse. Einfach mit Index [-1] direkt ansteuern. Dann beim Tag *li* direkt die *class='adress-item'* ansteuern, und zwar das erste Element *title. 

Mit Plottis Hilfe sieht das dann so aus:

In [72]:
soup.find_all("ol")[-1].find_all("li", {"class": "address-item"})[0]["title"]

'Dufourstr. 169,8008 Zürich,ZH'

Der Status der Berufsausübungsbewilligung ist im Tag h4. 

In [35]:
status = soup.find('h4').find("a")["title"].split(",")
(status[0] + status[1]).split("=")[0]

'Erteilt aktiv '

Oder vielleicht auch separat?


In [75]:
status = soup.find('h4').find("a")["title"].split(",")
status[0]

'Erteilt'

In [81]:
status[1].split(' = ')[0]

' aktiv'

Wie kommen wir an die Berufs- und Ausbildungsdaten?

In [102]:
soup.find('tbody')

<tbody>
<tr>
<td>Ärztin/Arzt</td>
<td>1969</td>
<td>Deutschland</td>
<td>Anerkennung<a class="info" title="Datum der Anerkennung des ausländischen Titels durch die Schweiz: Datum nach 1.6.2002 Anerkennung für EU/EFTA-Bürger durch die Bundesbehörden. Datum vor 1.6.2002 Bestätigung durch den Kanton."> </a>
                    (13.08.2002)
                  </td>
</tr>
<tr>
<th class="in-table-head">Weiterbildungstitel</th>
<th></th>
<th></th>
<th></th>
</tr>
<tr>
<td>Psychiatrie und Psychotherapie</td>
<td>2002</td>
<td>Schweiz</td>
<td>Eidgenössischer Weiterbildungstitel</td>
</tr>
<tr>
<th class="in-table-head">Weitere Qualifikationen (privatrechtliche Weiterbildung)</th>
<th></th>
<th></th>
<th></th>
</tr>
<tr>
<td colspan="4">Keine Angaben vorhanden</td>
</tr>
</tbody>

Ok, endlich gefunden. Mit dem Tag "tbody" komme ich an die weiteren Daten. Wir versuchen uns weiter durch die durch td- und tr-Tags getrennten Daten zu kämpfen. 

In [103]:
berufsdaten = soup.find('tbody')

In [110]:
beruf = berufsdaten.find('td')

<td>Ärztin/Arzt</td>

In [113]:
soup.find('tbody').find('td').text

'Ärztin/Arzt'

In [117]:
soup.find('tbody').find('td').find_next('td').text

'1969'

In [118]:
soup.find('tbody').find('td').find_next('td').find_next('td').text

'Deutschland'

In [36]:
datum_string = soup.find('tbody').find('td').find_next('td').find_next('td').find_next('td').text

Das Datum der Anerkennung. Mit RegEx kriegen wir das hoffentlich sauber raus.

In [47]:
datum_string

'Anerkennung\xa0\n                    (13.08.2002)\n                  '

In [57]:
import datetime
date_time_str = re.findall(r'\d+.\d+.\d+', datum_string)[0]
date_time_obj = datetime.datetime.strptime(date_time_str, '%d.%m.%Y')
date_time_obj.strftime("%d.%m.%Y")

'13.08.2002'

Ok, auch hier half Plotti mit. Regex und Datetime generieren einen Wert im richtigen Format. 

Der Rest kommt sauber rein.

In [132]:
soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td').text

'Psychiatrie und Psychotherapie'

In [133]:
soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td').find_next('td').text

'2002'

In [136]:
soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td').find_next('td').find_next('td').text

'Schweiz'

ginge das auch so?

In [86]:
soup.find('tbody').find_all('tr')[-3].find('td')

<td>Psychiatrie und Psychotherapie</td>

Die Nationalitätsfrage ist noch ungelöst:
    

In [43]:
nationalität = soup.find('p').text#.find('Nationalität:')
nationalität


'Nationalität: Deutschland (DE)Sprachkenntnisse: DeutschGLN: 7601000677815\xa0UID: CHE103519026\xa0'

Mit Regex integriert:

In [44]:
nationalität = soup.find('p', text=re.compile("Nationalität:(.*)Sprachkenntnisse"))
nationalität


Kein Resultat. Hm. 

In [47]:
nationalität = soup.find('p').text(re.compile("Nationalität:(.*)Sprachkenntnisse"))
nationalität

TypeError: 'str' object is not callable

Oder ganz ohne Soup. Direkt mit Regex?

In [50]:
nationalität = re.findall("Nationalität:(.*)")
nationalität


TypeError: findall() missing 1 required positional argument: 'string'

Funktioniert nicht richtig.

In [59]:
nationalität = soup.find_all('p', text = re.compile("Nationalität:(.*)"))
nationalität

[]

In [63]:
nationalität = soup.find_all('p', text = re.compile("Nationalität:\\W+(\\w+)")


SyntaxError: unexpected EOF while parsing (<ipython-input-63-b9520cb4a416>, line 4)

Jetzt einmal testen mit Regex. 

In [77]:
import re

text = 'Nationalität: Deutschland (DE)Sprachkenntnisse: DeutschGLN: 7601000677815\xa0UID: CHE103519026\xa0'
nationalität = re.search('Nationalität: \w+', text)

nationalität


<re.Match object; span=(0, 25), match='Nationalität: Deutschland'>

Über eine Stunde vergebens rumgesucht und noch immer keine passende Regex-Formel gefunden. 

Jetzt das ganze in einem Codebrocken, noch für ein File (Zeile 11). Das wird die eigentliche Parsing-Funktion. 

In [17]:
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import re
import datetime
from tqdm import tqdm

path = 'MedReg_Scraped_Data_20K/'
file_list = os.listdir(path)

file = open(path + file_list[4805], 'r')
text = file.read()
soup = BeautifulSoup(text, 'html.parser')

def get_gender(soup):
    img_source = soup.find('h3').find("img")["src"]
    if "venus" in img_source:
        return "w"
    else:
        return "m"


name = soup.find('h3').text.replace('\xa0','')
#nationalität = soup.find('p')
geschlecht = get_gender(soup)
adresse = soup.find_all("ol")[-1].find_all("li", {"class": "address-item"})[0]["title"]
beruf = soup.find('tbody').find('td').text
jahr_diplom = soup.find('tbody').find('td').find_next('td').text
land_diplom = soup.find('tbody').find('td').find_next('td').find_next('td').text
datum_anerkennung = soup.find('tbody').find('td').find_next('td').find_next('td').find_next('td').text
weiterbildungstitel = soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td').text
jahr_wbtitel = soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td').find_next('td').text
land_wbtitel = soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td').find_next('td').find_next('td').text



#Umwandlung des Anerkennungsdatum später dann mit der Formel:
#date_time_str = re.findall(r'\d+.\d+.\d+', datum_anerkennung)[0]
#date_time_obj = datetime.datetime.strptime(date_time_str, '%d.%m.%Y')
#date_time_obj.strftime("%d.%m.%Y")



In [11]:
beruf

'Ärztin/Arzt'

In [12]:
adresse

'Dufourstr. 169,8008 Zürich,ZH'

In [13]:
weiterbildungstitel

'Psychiatrie und Psychotherapie'

In [19]:
date_time_obj

datetime.datetime(2002, 8, 13, 0, 0)

Die Daten scheinen jetzt einigermassen sauber ausgelesen zu werden.
Als nächstes müssen jetzt diese Daten reihenweise ausgelesen werden und in einem Dataframe abgelegt werden. Dazu erstelle ich ein das Notebook "MedReg Parsing seriell".